In [17]:
import respy as rp
import pandas as pd
import numpy as np

from auxiliary import *

pd.set_option('display.max_rows', 1000)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [51]:
params, options, data = rp.get_example_model("kw_97_basic")

num_occupations, n_periods = 3, 3

params_occ, options_occ = update_model_specification(params, options, num_occupations)

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


In [52]:
params_occ.loc[("shocks_sdcorr", slice(None)), :]

value  sd comment
category      name                                                 
shocks_sdcorr sd_aa                              1.0000 NaN     NaN
              sd_bb                              1.0000 NaN     NaN
              sd_blue_collar                     0.3329 NaN     NaN
              sd_cc                              1.0000 NaN     NaN
              sd_military                        0.3308 NaN     NaN
              sd_white_collar                    0.3301 NaN     NaN
              sd_school                       2312.0000 NaN     NaN
              sd_home                        13394.0000 NaN     NaN
              corr_bb_aa                         0.0000 NaN     NaN
              corr_blue_collar_aa                0.0000 NaN     NaN
              corr_blue_collar_bb                0.0000 NaN     NaN
              corr_cc_aa                         0.0000 NaN     NaN
              corr_cc_bb                         0.0000 NaN     NaN
              corr_cc_blue_collar                0.0000 NaN     NaN
              corr_military_aa                   0.0000 NaN     NaN
              corr_military_bb                   0.0000 NaN     NaN
              corr_military_blue_collar          0.4120 NaN     NaN
              corr_military_cc                   0.0000 NaN     NaN
              corr_white_collar_aa               0.0000 NaN     NaN
              corr_white_collar_bb               0.0000 NaN     NaN
              corr_white_collar_blue_collar     -0.3806 NaN     NaN
              corr_white_collar_cc               0.0000 NaN     NaN
              corr_white_collar_military        -0.3688 NaN     NaN
              corr_school_aa                     0.0000 NaN     NaN
              corr_school_bb                     0.0000 NaN     NaN
              corr_school_blue_collar            0.0000 NaN     NaN
              corr_school_cc                     0.0000 NaN     NaN
              corr_school_military               0.0000 NaN     NaN
              corr_school_white_collar           0.0000 NaN     NaN
              corr_home_aa                       0.0000 NaN     NaN
              corr_home_bb                       0.0000 NaN     NaN
              corr_home_blue_collar              0.0000 NaN     NaN
              corr_home_cc                       0.0000 NaN     NaN
              corr_home_military                 0.0000 NaN     NaN
              corr_home_white_collar             0.0000 NaN     NaN
              corr_home_school                   0.0000 NaN     NaN